In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected
import torch_geometric.transforms as T
from torch_geometric.utils import sort_edge_index
from torch_geometric.loader import LinkNeighborLoader
import itertools
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from infrastructure.repositories import PyGDataRepository
from config.settings import MAPPING_PATH, PYG_DATA_PATH, OUTPUT_DIM, TRAINING_HISTORY_PATH, MODEL_PATH, PYG_TRAINING_DATA_PATH
torch.set_float32_matmul_precision('medium')
from torch.cuda.amp import GradScaler, autocast

# Architecture

# Function to train

In [ ]:
import os
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected
import torch_geometric.transforms as T
from torch_geometric.utils import sort_edge_index
from torch_geometric.loader import LinkNeighborLoader
import itertools
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from infrastructure.repositories import PyGDataRepository
from config.settings import  PYG_DATA_PATH, OUTPUT_DIM, TRAINING_HISTORY_PATH, MODEL_PATH, \
    PYG_TRAINING_DATA_PATH
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
torch.set_float32_matmul_precision('medium')
from torch.cuda.amp import GradScaler, autocast


# Architecture
class GraphSAGE(torch.nn.Module):
    def __init__(self,hidden_channels, out_channels, dropout):
        super().__init__()
        self.conv1 = SAGEConv((-1,-1), hidden_channels)
        self.ln1 = torch.nn.LayerNorm(hidden_channels)
        self.conv2 = SAGEConv((-1,-1), out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index):

        x = self.conv1(x, edge_index)
        x = self.ln1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        return x

class InteractionMLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        # Input dim * 3 vì ta sẽ nối [src, dst, src*dst]
        self.lin1 = torch.nn.Linear(input_dim * 3, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, hidden_dim // 2)
        self.lin3 = torch.nn.Linear(hidden_dim // 2, output_dim)

        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, z_src, z_dst):
        # TẠO TƯƠNG TÁC MẠNH MẼ
        # 1. Đặc trưng gốc: z_src, z_dst
        # 2. Đặc trưng tương đồng: z_src * z_dst (Hadamard product)
        # Giúp model dễ dàng học được "A và B có giống nhau không?"
        combined = torch.cat([z_src, z_dst, z_src * z_dst], dim=1)

        h = self.lin1(combined)
        h = F.relu(h)
        h = self.dropout(h)

        h = self.lin2(h)
        h = F.relu(h)
        h = self.dropout(h)

        h = self.lin3(h)

        # Ép về 1 chiều để tránh lỗi shape [N, 1] vs [N]
        return h.view(-1)
import torch
import torch.nn.functional as F
from torch_geometric.nn import HGTConv, Linear

class HGTLinkPrediction(torch.nn.Module):
    def __init__(self,  hidden_channels, out_channels,data, dropout=0.5, num_heads=4, num_layers=3):
        super().__init__()

        # 1. INPUT PROJECTION (Quan trọng)
        # Biến đổi vector Text (768 dim) + Year (1 dim) về không gian chung (256 dim)
        # Giúp model học được đặc trưng riêng cho bài toán này
        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            # Lấy kích thước feature đầu vào thực tế từ data
            in_dim = data[node_type].x.size(1)
            self.lin_dict[node_type] = Linear(in_dim, hidden_channels)

        # 2. HGT LAYERS (Thay cho SAGE)
        # HGT dùng cơ chế Attention để "đọc hiểu" feature text tốt hơn SAGE
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           heads=num_heads)
            self.convs.append(conv)

        # 3. INTERACTION DECODER (Như đã bàn)
        self.decoders = torch.nn.ModuleDict()
        unique_rels = set()
        for src, rel, dst in data.edge_types:
            if not rel.startswith('rev_'): # Bỏ qua cạnh ngược
                unique_rels.add(rel)

        for rel in unique_rels:
            key = f"__{rel}__"
            self.decoders[key] = InteractionMLP(out_channels, 64, 1, dropout)

    def forward(self, x_dict, edge_index_dict, target_edge_type, edge_label_index):
        # A. Projection: Ép feature text + year vào không gian Hidden
        dtype = self.kqv_lin.weights[0].dtype if hasattr(self.kqv_lin, 'weights') else torch.float32
        x_dict = {k: v.to(dtype) for k, v in x_dict.items()}
        x_start = {}
        for node_type, x in x_dict.items():
            x_start[node_type] = self.lin_dict[node_type](x).relu_()
            x_start[node_type] = self.dropout(x_start[node_type])

        # B. HGT Message Passing
        for conv in self.convs:
            x_start = conv(x_start, edge_index_dict)

        # C. Decode
        src_type, rel, dst_type = target_edge_type
        z_src = x_start[src_type][edge_label_index[0]]
        z_dst = x_start[dst_type][edge_label_index[1]]

        key = f"__{rel}__"
        if key in self.decoders:
            return self.decoders[key](z_src, z_dst)
        else:
            return torch.zeros(z_src.size(0), device=z_src.device)

class LinkPredictionModel(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, data=None, dropout=0.5):
        super().__init__()

        # Dùng DeepGraphSAGE thay vì bản thường
        self.gnn = GraphSAGE(hidden_channels, out_channels, dropout)
        self.encoder = to_hetero(self.gnn, data.metadata(), aggr='sum')

        self.decoders = torch.nn.ModuleDict()

        # Init Decoder cho từng loại cạnh
        unique_rels = set()
        for src, rel, dst in data.edge_types:
            if not rel.startswith('rev_'): # Bỏ qua cạnh ngược
                unique_rels.add(rel)

        for rel in unique_rels:
            key = f"__{rel}__"
            self.decoders[key] = InteractionMLP(out_channels, 64, 1, dropout)

    def forward(self, x, edge_index, target_edge_type, edge_label_index):
        z_dict = self.encoder(x, edge_index)

        src_type, rel, dst_type = target_edge_type

        z_src = z_dict[src_type][edge_label_index[0]]
        z_dst = z_dict[dst_type][edge_label_index[1]]

        key = f"__{rel}__"

        if key in self.decoders:
            return self.decoders[key](z_src, z_dst)
        else:
            return torch.zeros(z_src.size(0), device=z_src.device)

# Function to train
import optuna

from config.settings import BATCH_SIZE, MODEL_PATH
from infrastructure.repositories import ModelRepository
import gc



def get_or_prepare_data():
    """Tải và chuẩn bị dữ liệu (Undirected + Sanitize)."""
    feature_repo = PyGDataRepository(PYG_DATA_PATH)
    data = feature_repo.load_data()

    if data is None:
        print("Chưa có dữ liệu PyG. Vui lòng chạy ETL trước!")
        return None

    # Xóa cạnh rỗng
    #data = sanitize_hetero_data(data)

    return data


def loader_generator(data_source, target_edge_types, batch_size, shuffle=False):
    for et in target_edge_types:
        # 1. Kiểm tra nhanh (giữ nguyên logic cũ của bạn)
        if et not in data_source.edge_index_dict: pass
        if hasattr(data_source[et], 'edge_label_index') and data_source[et].edge_label_index.numel() == 0:
            continue

        # 2. Chuẩn bị nhãn
        lbl_index = data_source[et].edge_label_index
        lbl_ones = torch.ones(lbl_index.size(1), dtype=torch.float32)

        # 3. Khởi tạo Loader (Chỉ tốn RAM tại thời điểm này)

        loader = LinkNeighborLoader(
            data_source,
            num_neighbors=[50, 30],
            edge_label_index=(et, lbl_index),
            edge_label=lbl_ones,
            neg_sampling_ratio=3.0,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=0
        )
        # 4. Trả về để dùng ngay
        yield et, loader


def train_epoch(model, data, optimizer, device, target_edge_types, scaler, batch_size=BATCH_SIZE):
    model.train()
    total_loss = 0
    total_examples = 0
    count = 1
    max = len(target_edge_types)
    data_loader_gen = loader_generator(data, target_edge_types, batch_size, shuffle=True)
    for edge_type, loader in data_loader_gen:
        pbar = tqdm(loader, desc="Training", leave=False)
        pbar.set_postfix({
            "relationship": f" {edge_type[1]} | {count}/{max}"
        })
        for batch in pbar:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Với single-type loader, ta biết chắc chắn cạnh cần dự đoán là edge_type
            # PyG tự động gán vào edge_label_index của edge_type đó trong batch
            edge_label_index = batch[edge_type].edge_label_index
            edge_label = batch[edge_type].edge_label
            with torch.amp.autocast('cuda'):
                # Forward
                out = model(batch.x_dict, batch.edge_index_dict, edge_type, edge_label_index)

                # Loss
                loss = F.binary_cross_entropy_with_logits(out, edge_label)

            scaler.scale(loss).backward()

            # 3. Optimizer Step
            scaler.step(optimizer)
            scaler.update()  # Cập nhật lại hệ số scale cho lần sau

            total_loss += loss.item() * edge_label.size(0)
            total_examples += edge_label.size(0)
        count += 1
        del loader, pbar
        gc.collect()
    return total_loss / (total_examples + 1e-6)


@torch.no_grad()
def evaluate(model, data, device, target_edge_types, batch_size=BATCH_SIZE):
    """
    Đánh giá mô hình trên tập Val hoặc Test.

    Args:
        model: Mô hình GNN đã huấn luyện.
        loaders: List các LinkNeighborLoader (mỗi loader ứng với 1 loại cạnh).
        device: 'cuda' hoặc 'cpu'.
        target_edge_types: List các loại cạnh tương ứng với loaders.

    Returns:
        score: Chỉ số ROC-AUC (0.0 -> 1.0).
    """
    model.eval()  # Chuyển model sang chế độ đánh giá (tắt Dropout, khóa BatchNorm)

    preds = []
    ground_truths = []

    # 1. Duyệt song song qua từng cặp (Loại cạnh, Loader tương ứng)
    # Lưu ý: target_edge_types và loaders phải có cùng độ dài và thứ tự
    count = 1
    max = len(target_edge_types)
    data_loader_gen = loader_generator(data, target_edge_types, batch_size, shuffle=False)
    for edge_type, loader in data_loader_gen:

        pbar = tqdm(loader, desc="Validation", leave=False)
        pbar.set_postfix({
            "relationship": f" {edge_type[1]} | {count}/{max}"
        })
        for batch in pbar:
            batch = batch.to(device)

            # Kiểm tra an toàn: Batch có chứa nhãn cho loại cạnh này không?
            if not hasattr(batch[edge_type], 'edge_label_index') or batch[edge_type].edge_label_index.numel() == 0:
                continue

            # Lấy dữ liệu "đề thi"
            edge_label_index = batch[edge_type].edge_label_index
            edge_label = batch[edge_type].edge_label

            # 3. Forward Pass
            # Truyền đúng edge_type để model biết dùng trọng số nào (nếu có chia tách)
            # Output model thường là Logits (chưa qua Sigmoid)
            with torch.amp.autocast('cuda'):
                out = model(batch.x_dict, batch.edge_index_dict, edge_type, edge_label_index)
                # Sigmoid cũng nên nằm trong context này (hoặc không, tùy ý, nhưng forward model bắt buộc phải có)
                out = torch.sigmoid(out)

            # 4. Lưu lại kết quả (Đưa về CPU và Numpy để tính toán bằng Sklearn)
            preds.append(out.cpu().numpy())
            ground_truths.append(edge_label.cpu().numpy())
        count += 1
        del loader, pbar
    # 5. Xử lý trường hợp không có dữ liệu (tránh lỗi crash)
    if len(preds) == 0:
        print("Not data for validation.")
        return 0.0

    # 6. Gộp tất cả các mảng numpy lại thành 1 mảng dài duy nhất
    final_preds = np.concatenate(preds)
    final_labels = np.concatenate(ground_truths)

    if np.isnan(final_preds).any():
        print("❌ LỖI NGHIÊM TRỌNG: Model output chứa NaN!")
        return 0.0

    # Kiểm tra xem Labels có đủ 2 lớp (0 và 1) không
    unique_labels = np.unique(final_labels)
    if len(unique_labels) < 2:
        print(f"⚠️ CẢNH BÁO: Tập Label chỉ chứa 1 loại nhãn duy nhất: {unique_labels}")
        print("-> Lý do: Loader Validation chưa bật 'neg_sampling_ratio'!")
        return 0.0

    # 7. Tính ROC-AUC Score
    try:
        return roc_auc_score(final_labels, final_preds)
    except ValueError as e:
        print(f"❌ SKLEARN ERROR: {e}")
        # In thêm thống kê để biết tại sao
        print(f"Min Pred: {final_preds.min()}, Max Pred: {final_preds.max()}")
        print(f"Unique Labels: {np.unique(final_labels)}")
        return 0.0


# --- 3. CHIẾN LƯỢC CHẠY ---
def get_edge_pairs(data):
    """
    Tự động bắt cặp cạnh thuận và cạnh nghịch.
    Quy tắc: Cạnh nghịch có thêm tiền tố 'rev_' hoặc là chiều ngược lại.
    """
    forward_edges = []
    reverse_edges = []

    for edge_type in data.edge_types:
        src, rel, dst = edge_type

        # 1. Bỏ qua nếu đây là cạnh 'rev_' (chúng ta sẽ xử lý nó khi gặp cạnh thuận)
        if rel.startswith('rev_'):
            continue
        if src != 'human' or dst != 'human':
            continue
        # 2. Xây dựng tên cạnh ngược dự kiến
        rev_rel = f"rev_{rel}"
        rev_edge_type = (dst, rev_rel, src)

        # 3. Kiểm tra xem cạnh ngược này có tồn tại trong data không
        if rev_edge_type in data.edge_types:
            forward_edges.append(edge_type)
            reverse_edges.append(rev_edge_type)

    return forward_edges, reverse_edges


def prepare_data_splits(data, val_ratio=0.1, test_ratio=0.1):
    """
    Sử dụng RandomLinkSplit chuẩn của PyG.
    """
    print("--- PREPARING DATA SPLITS (RandomLinkSplit) ---")

    # 1. Tự động bắt cặp cạnh để xử lý Leakage
    target_edge_types, rev_edge_types = get_edge_pairs(data)
    for edge in target_edge_types:
        print(edge)
    print(f"-> Target Edges (Predicting): {len(target_edge_types)} types")

    # 2. Cấu hình Splitter
    transform = T.RandomLinkSplit(
        num_val=val_ratio,
        num_test=test_ratio,
        is_undirected=False,  # Hetero Graph có hướng

        edge_types=target_edge_types,
        rev_edge_types=rev_edge_types,

        # Tách 20% cạnh train ra làm "Label" (Supervision), 80% giữ lại nối dây
        disjoint_train_ratio=0.2,

        add_negative_train_samples=False
    )

    # 3. Thực hiện chia (Tạo ra 3 object Data riêng biệt)
    train_data, val_data, test_data = transform(data)

    return train_data, val_data, test_data, target_edge_types


def call_back(model):
    repo = ModelRepository(MODEL_PATH)
    repo.save_model(model)

def update_training_history(file_path, new_data):
    # Bước 1: Khởi tạo dữ liệu hiện có
    history = []

    # Bước 2: Kiểm tra và Đọc dữ liệu cũ
    if os.path.exists(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                history = json.load(f)
                # Đảm bảo history là một list để có thể append
                if not isinstance(history, list):
                    history = [history]
        except (json.JSONDecodeError, ValueError):
            # Xử lý nếu file trống hoặc lỗi định dạng
            history = []

    # Bước 3: Cập nhật dữ liệu mới
    if isinstance(new_data, list):
        history.extend(new_data)
    else:
        history.append(new_data)

    # Bước 4: Ghi đè lại toàn bộ file với mode "w"
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(history, f, indent=4, ensure_ascii=False)
def train_one_config(split_data, config, device, final_mode=False, trial=None):
    print("Initializing Data for Training!", flush=True, end=" ")
    train_data, val_data, test_data, target_edge_types = split_data
    print("Completed!", flush=True)
    hidden_dim = config['hidden_dim']
    lr = config['lr']
    epochs = config['epochs']
    batch_size = config['batch_size']
    dropout = config['dropout']
    print("Initializing Model!", flush=True, end=" ")
    model = LinkPredictionModel(hidden_dim, OUTPUT_DIM, train_data, dropout).to(device)
    state_dict = torch.load(MODEL_PATH,weights_only= False, map_location=device)
    model.load_state_dict(state_dict)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    print("Completed!", flush=True)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=5,      # Reset LR mỗi 5 epoch
        T_mult=2,   # Lần sau dài gấp đôi (5 -> 10 -> 20)
        eta_min=1e-5 # LR thấp nhất
    )

    early_stop_patience = 10
    early_stop_counter = 0
    history = {"epoch": [], "loss": [], "val_auc": []}
    best_val_auc = 0.7485
    final_test_auc = 0.0
    best_model_state = None
    best_epoch_found = 0

    print(f"\nHidden={hidden_dim}, LR={lr}, epochs={epochs}, dropout={dropout}, batch_size={batch_size}")
    scaler = torch.amp.GradScaler('cuda')  # 4. TRAINING LOOP
    for epoch in range(1, epochs + 1):
        loss = train_epoch(model, train_data, optimizer, device, target_edge_types, scaler, batch_size)

        history["epoch"].append(epoch)
        history["loss"].append(float(loss))
        log_msg = f"Epoch {epoch:03d} | Loss: {loss:.4f}"
        torch.cuda.empty_cache()
        gc.collect()
        # Đánh giá trên tập Val để chọn Model tốt nhất
        val_auc = evaluate(model, val_data, device, target_edge_types, batch_size)
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        history["val_auc"].append(float(val_auc))
        log_msg += f" | Val AUC: {val_auc:.4f}| LR: {current_lr :.5f}"
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_model_state = model.state_dict()  # Lưu lại state tốt nhất
            early_stop_counter = 0
            call_back(model)
        else:
            if val_auc == best_val_auc:
                early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print(
                    f"Early Stopping tại Epoch {epoch} vì Val AUC không giao động trong {early_stop_patience} epochs.")
                break
        update_training_history(TRAINING_HISTORY_PATH,history)
        print(log_msg)

    # 5. ĐÁNH GIÁ CUỐI CÙNG TRÊN TẬP TEST
    if not final_mode and best_model_state:
        # Load lại trọng số tốt nhất (đạt đỉnh ở Val) để test
        model.load_state_dict(best_model_state)
        final_test_auc = evaluate(model, test_data, device, target_edge_types, batch_size)
        history['test_auc'] = final_test_auc
        print(f"--> Test AUC: {final_test_auc:.4f}")

    update_training_history(TRAINING_HISTORY_PATH,history)

    # Nếu là final mode thì trả về 1.0 (hoặc train auc)
    best_model = model
    final_test_auc = 1.0
    best_epoch_found = epochs

    # Trả về Test AUC thay vì Val AUC để Grid Search in ra kết quả thực tế hơn
    # Hoặc bạn vẫn có thể trả về Val AUC để chọn tham số, nhưng in Test AUC để tham khảo

    torch.cuda.empty_cache()  # Xả VRAM
    gc.collect()
    return best_val_auc, final_test_auc, best_model



# --- PHẦN SCRIPT TEST (TEST BENCH) ---

def run_test(data):
    l = torch.tensor([is_edge_index_sorted(data[edge_type].edge_index) for edge_type in data.edge_types])
    print(l.all())


def pre_process_data(data):
    # Giả sử data là HeteroData
    data.pin_memory()  # Tăng tốc transfer dữ liệu
    return data


def run_optimization(data = None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on: {device}")

    # --- BƯỚC 1: CHUẨN BỊ DATA CHO GRID SEARCH ---
    print("\n>>> Loading & Splitting Data...")

    # 1. Load dữ liệu gốc
    #if data is None:
    data = get_or_prepare_data()

    # 2. Cắt dữ liệu (In-Place Modification)
    # Hàm này sẽ xóa cạnh Val/Test khỏi `data` và trả về indices rời
    # train_graph chính là biến `data` sau khi bị cắt
    split_data = prepare_data_splits(data, val_ratio=0.001, test_ratio=0.001)
    config = {
        'hidden_dim': 256,
        'batch_size': 64,
        'lr': 0.01,
        'dropout': 0.41372892081262375,
        'epochs': 1
    }
    best_val_auc, test_auc, best_model = train_one_config(split_data, config, device)

    repo = ModelRepository(MODEL_PATH)
    repo.save_model(best_model)


# Loading data
#data = ToUndirected(merge=False)(data)
#feature.save_data(data,mapping)
# Training
if __name__ == "__main__":
    run_optimization()


# PHASE 2

In [ ]:
from segmentation_models_pytorch.metrics import f1_score
import os
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader
import json
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score
from infrastructure.repositories import PyGDataRepository, ModelRepository
from config.settings import PYG_DATA_PATH, OUTPUT_DIM, TRAINING_HISTORY_PATH, MODEL_PATH
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import gc

torch.set_float32_matmul_precision('medium')

# --- 1. ARCHITECTURE (Giữ nguyên như cũ) ---
class GraphSAGE(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, dropout):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.ln1 = torch.nn.LayerNorm(hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.ln1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        return x

class InteractionMLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.lin1 = torch.nn.Linear(input_dim * 3, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, hidden_dim // 2)
        self.lin3 = torch.nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, z_src, z_dst):
        combined = torch.cat([z_src, z_dst, z_src * z_dst], dim=1)
        h = self.lin1(combined)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.lin2(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.lin3(h)
        return h.view(-1)

class LinkPredictionModel(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, data=None, dropout=0.5):
        super().__init__()
        self.gnn = GraphSAGE(hidden_channels, out_channels, dropout)
        self.encoder = to_hetero(self.gnn, data.metadata(), aggr='sum')
        self.decoders = torch.nn.ModuleDict()

        unique_rels = set()
        for src, rel, dst in data.edge_types:
            if not rel.startswith('rev_'):
                unique_rels.add(rel)

        for rel in unique_rels:
            key = f"__{rel}__"
            self.decoders[key] = InteractionMLP(out_channels, 64, 1, dropout)

    def forward(self, x, edge_index, target_edge_type, edge_label_index):
        z_dict = self.encoder(x, edge_index)
        src_type, rel, dst_type = target_edge_type
        z_src = z_dict[src_type][edge_label_index[0]]
        z_dst = z_dict[dst_type][edge_label_index[1]]

        key = f"__{rel}__"
        if key in self.decoders:
            return self.decoders[key](z_src, z_dst)
        else:
            return torch.zeros(z_src.size(0), device=z_src.device)

# --- 2. DATA UTILS ---

def get_or_prepare_data():
    feature_repo = PyGDataRepository(PYG_DATA_PATH)
    data = feature_repo.load_data()
    return data

def get_human_hub_edges(data):
    """
    [PHASE 2] Lấy các cạnh Human-Hub để Calibration.
    """
    target_edges = []
    rev_target_edges = []
    # Danh sách Hubs quan trọng để hiệu chỉnh Encoder
    calibration_rels = [
        ('human', 'educated_at', 'organization'),
        ('human', 'educated_at', 'educational_institution'),
        ('human','acted_in','film'),
        ('human', 'work_at', 'educational_institution'),
        ('human', 'work_at', 'organization'),
        ('human', 'award_received', 'awards')
    ]

    print("\n--- SELECTING HUB EDGES FOR CALIBRATION ---")
    for edge_type in data.edge_types:
        src, rel, dst = edge_type
        if rel.startswith('rev_'):
            continue
        rev_rel = f"rev_{rel}"
        if src == 'human' and dst != 'human':
            if (src,rel,dst) in calibration_rels:
                target_edges.append(edge_type)
                rev_target_edges.append((dst, rev_rel, src))
                print(f"Selected: {edge_type}")


    return target_edges, rev_target_edges

def prepare_calibration_splits(data):
    """
    Split cho Phase 2: Dùng Human-Hub làm Label.
    Nhưng vẫn giữ nguyên cấu trúc đồ thị.
    """
    target_edges, rev_target_edges = get_human_hub_edges(data)

    if len(target_edges) == 0: raise ValueError("No hub edges found!")

    transform = T.RandomLinkSplit(
        num_val=0.01,  # Chỉ cần val ít để check trend
        num_test=0.01,
        is_undirected=False,
        edge_types=target_edges,
        rev_edge_types=rev_target_edges,
        disjoint_train_ratio=0.1, # Cắt 10% Hub ra làm bài tập hiệu chỉnh
        add_negative_train_samples=False
    )

    train_data, val_data, test_data = transform(data)
    return train_data, val_data, test_data, target_edges

def loader_generator(data_source, target_edge_types, batch_size, shuffle=False):
    for et in target_edge_types:
        if et not in data_source.edge_index_dict: continue
        if hasattr(data_source[et], 'edge_label_index') and data_source[et].edge_label_index.numel() == 0:
            continue

        lbl_index = data_source[et].edge_label_index
        lbl_ones = torch.ones(lbl_index.size(1), dtype=torch.float32)

        loader = LinkNeighborLoader(
            data_source,
            num_neighbors=[50, 30],
            edge_label_index=(et, lbl_index),
            edge_label=lbl_ones,
            neg_sampling_ratio=3.0,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=0,
            persistent_workers=False
        )
        yield et, loader

# --- 3. TRAIN & EVALUATE ---

def train_epoch(model, data, optimizer, device, target_edge_types, scaler, batch_size):
    model.train() # ENCODER LÚC NÀY ĐANG MỞ (UNFROZEN)
    total_loss = 0
    total_examples = 0

    data_loader_gen = loader_generator(data, target_edge_types, batch_size, shuffle=True)

    for edge_type, loader in data_loader_gen:
        pbar = tqdm(loader, desc=f"Calibrating {edge_type[1]}", leave=False)
        for batch in pbar:
            batch = batch.to(device)
            optimizer.zero_grad()

            edge_label_index = batch[edge_type].edge_label_index
            edge_label = batch[edge_type].edge_label

            with torch.amp.autocast('cuda'):
                # Forward qua Encoder (đang tính gradient)
                out = model(batch.x_dict, batch.edge_index_dict, edge_type, edge_label_index)
                loss = F.binary_cross_entropy_with_logits(out, edge_label)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item() * edge_label.size(0)
            total_examples += edge_label.size(0)

        del loader, pbar

    return total_loss / (total_examples + 1e-6)

@torch.no_grad()
def evaluate(model, data, device, target_edge_types, batch_size):
    model.eval()
    preds, ground_truths = [], []

    data_loader_gen = loader_generator(data, target_edge_types, batch_size, shuffle=False)
    for edge_type, loader in data_loader_gen:
        for batch in loader:
            batch = batch.to(device)
            if not hasattr(batch[edge_type], 'edge_label_index') or batch[edge_type].edge_label_index.numel() == 0: continue

            with torch.amp.autocast('cuda'):
                out = model(batch.x_dict, batch.edge_index_dict, edge_type, batch[edge_type].edge_label_index)
                out = torch.sigmoid(out)

            preds.append(out.cpu().numpy())
            ground_truths.append(batch[edge_type].edge_label.cpu().numpy())

    if len(preds) == 0: return 0.0
    final_labels = np.concatenate(ground_truths)
    final_preds = np.concatenate(preds)
    ap = 0
    f1 = 0
    try:
        ap = average_precision_score(final_labels, final_preds)
        f1 = f1_score(final_labels, (final_preds > 0.5).astype(int))
    except:
        pass
    auc = roc_auc_score(final_labels, final_preds )
    return auc, ap, f1

def update_training_history(file_path, new_data):
    history = []
    if os.path.exists(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                history = json.load(f)
                if not isinstance(history, list): history = [history]
        except: history = []
    if isinstance(new_data, list): history.extend(new_data)
    else: history.append(new_data)
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(history, f, indent=4, ensure_ascii=False)

# --- 4. MAIN: PHASE 2 CALIBRATION ---

def run_phase2_calibration():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🚀 RUNNING PHASE 2: CALIBRATION (UNFROZEN ENCODER) on {device}")

    # 1. Load Data
    data = get_or_prepare_data()

    # 2. Split Data (Dùng Human-Hub làm bài tập)
    train_data, val_data, test_data, target_edges = prepare_calibration_splits(data)

    # Cấu hình Phase 2
    config = {
        'hidden_dim': 256,
        'batch_size': 64,
        'lr': 1e-5,
        'epochs': 1,
        'dropout': 0.4
    }

    # 3. Init Model
    print("Initializing Model...", end=" ")
    model = LinkPredictionModel(config['hidden_dim'], OUTPUT_DIM, train_data, config['dropout']).to(device)

    # 4. Load Phase 1 Weights
    print("\n📥 Loading Phase 1 (Social) Weights...")
    try:
        # Load weights, bỏ qua strict để an toàn
        state_dict = torch.load(MODEL_PATH, map_location=device, weights_only=False)
        model.load_state_dict(state_dict, strict=False)
        print("Phase 1 Weights Loaded.")
    except Exception as e:
        print(f"Error loading weights: {e}")

    print("Encoder is UNFROZEN. Learning Rate set to 1e-5 to preserve knowledge.")

    # Lấy param của Encoder
    params = list(model.encoder.parameters())

    added_keys = set()

    for src, rel, dst in target_edges:
        key = f"__{rel}__"
        if key in model.decoders and key not in added_keys:
            params.extend(list(model.decoders[key].parameters()))
            added_keys.add(key) # Đánh dấu là đã thêm

    optimizer = torch.optim.Adam(params, lr=config['lr'])
    scaler = torch.amp.GradScaler('cuda')

    # 6. Training Loop
    history = {"epoch": [], "loss": [], "val_auc": []}
    calib_model_path = str(MODEL_PATH).replace(".pt", "_phase2_calibrated.pt")
    calib_history_path = str(TRAINING_HISTORY_PATH).replace(".json", "_phase2.json")

    print(f"\nStart Calibration for {config['epochs']} epochs...")

    for epoch in range(1, config['epochs'] + 1):
        # Train (Trên cạnh Hubs)
        loss = train_epoch(model, train_data, optimizer, device, target_edges, scaler, config['batch_size'])

        # Validate (Trên cạnh Hubs - để xem tiến bộ)
        val_auc, ap, f1 = evaluate(model, val_data, device, target_edges, config['batch_size'])

        history["epoch"].append(epoch)
        history["loss"].append(loss)
        history["val_auc"].append(val_auc)
        try:
            history["ap"].append(ap)
            history["f1"].append(f1)
        except:
            pass
        print(f"Phase 2 - Epoch {epoch}: Loss={loss:.4f} | Hubs AUC={val_auc:.4f}")

        # Luôn save lại model sau epoch cuối (vì chỉ chạy 1-2 epoch nên không cần early stop)
        repo = ModelRepository(calib_model_path)
        repo.save_model(model)
    test_auc,ap , f1 = evaluate(model, test_data, device, target_edges, config['batch_size'])
    history["test_auc"].append(test_auc)
    try:
            history["test_ap"].append(ap)
            history["test_f1"].append(f1)
    except:
        pass
    print(f"Phase 2 Completed. Model saved to: {calib_model_path}")
    update_training_history(calib_history_path, history)
    return model
if __name__ == "__main__":
    model = run_phase2_calibration()

# PHASE 3

In [ ]:
import os
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero, HGTConv, Linear
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader
import json
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score, f1_score,average_precision_score
from infrastructure.repositories import PyGDataRepository, ModelRepository
from config.settings import PYG_DATA_PATH, OUTPUT_DIM, TRAINING_HISTORY_PATH, MODEL_PATH
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import gc

torch.set_float32_matmul_precision('medium')

# --- 1. ARCHITECTURE CLASSES (Giữ nguyên) ---
class GraphSAGE(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, dropout):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.ln1 = torch.nn.LayerNorm(hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.ln1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        return x

class InteractionMLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.lin1 = torch.nn.Linear(input_dim * 3, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, hidden_dim // 2)
        self.lin3 = torch.nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, z_src, z_dst):
        combined = torch.cat([z_src, z_dst, z_src * z_dst], dim=1)
        h = self.lin1(combined)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.lin2(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.lin3(h)
        return h.view(-1)

class LinkPredictionModel(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, data=None, dropout=0.5):
        super().__init__()
        self.gnn = GraphSAGE(hidden_channels, out_channels, dropout)
        self.encoder = to_hetero(self.gnn, data.metadata(), aggr='sum')
        self.decoders = torch.nn.ModuleDict()

        unique_rels = set()
        for src, rel, dst in data.edge_types:
            if not rel.startswith('rev_'):
                unique_rels.add(rel)

        for rel in unique_rels:
            key = f"__{rel}__"
            self.decoders[key] = InteractionMLP(out_channels, 64, 1, dropout)

    def forward(self, x, edge_index, target_edge_type, edge_label_index):
        # Encoder tính toán ra vector (Embedding)
        z_dict = self.encoder(x, edge_index)

        src_type, rel, dst_type = target_edge_type
        z_src = z_dict[src_type][edge_label_index[0]]
        z_dst = z_dict[dst_type][edge_label_index[1]]

        key = f"__{rel}__"
        if key in self.decoders:
            return self.decoders[key](z_src, z_dst)
        else:
            return torch.zeros(z_src.size(0), device=z_src.device)

# --- 2. DATA PREPARATION UTILS ---

def get_or_prepare_data():
    feature_repo = PyGDataRepository(PYG_DATA_PATH)
    data = feature_repo.load_data()
    if data is None:
        print("Chưa có dữ liệu PyG. Vui lòng chạy ETL trước!")
        return None
    return data

def get_recommendation_edges(data):
    """
    [MỚI] Chọn lọc các cạnh Human-Hub để train Decoder.
    """
    target_edges = []
    rev_target_edges = []

    print("\n--- SELECTING RECOMMENDATION EDGES ---")
    for edge_type in data.edge_types:
        src, rel, dst = edge_type
        if rel.startswith('rev_'):
            continue
        rev_rel = f"rev_{rel}"
        rev_edge_type = (dst, rev_rel, src)
        target_edges.append(edge_type)
        rev_target_edges.append(rev_edge_type)
        print(f"Selected Target: {edge_type} | Count: {data[edge_type].edge_index.size(1)}")

    return target_edges, rev_target_edges
def prepare_decoder_splits(data):
    """
    [MỚI] Split dữ liệu chuyên dụng cho Phase 3.
    """
    target_edges, rev_target_edges = get_recommendation_edges(data)

    if len(target_edges) == 0:
        raise ValueError("Không tìm thấy cạnh Recommendation nào trong dữ liệu!")

    print("--- SPLITTING DATA FOR DECODER TUNING ---")
    transform = T.RandomLinkSplit(
        num_val=0.05,  # Validation nhỏ thôi
        num_test=0.05, # Test nhỏ thôi
        is_undirected=False,
        edge_types=target_edges,
        rev_edge_types=rev_target_edges,

        # Chỉ lấy 20% cạnh Hub làm "đề thi", 80% giữ lại để Encoder có ngữ cảnh tốt
        disjoint_train_ratio=0.2,
        add_negative_train_samples=False
    )

    train_data, val_data, test_data = transform(data)
    return train_data, val_data, test_data, target_edges

def loader_generator(data_source, target_edge_types, batch_size, shuffle=False):
    for et in target_edge_types:
        if et not in data_source.edge_index_dict: continue
        if hasattr(data_source[et], 'edge_label_index') and data_source[et].edge_label_index.numel() == 0:
            continue

        lbl_index = data_source[et].edge_label_index
        lbl_ones = torch.ones(lbl_index.size(1), dtype=torch.float32)

        loader = LinkNeighborLoader(
            data_source,
            num_neighbors=[50, 30], # Giữ nguyên cấu hình tốt nhất
            edge_label_index=(et, lbl_index),
            edge_label=lbl_ones,
            neg_sampling_ratio=3.0,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=0,
            persistent_workers=False
        )
        yield et, loader

# --- 3. CORE FUNCTIONS (Train & Evaluate) ---

def train_epoch(model, data, optimizer, device, target_edge_types, scaler, batch_size):
    model.train()
    total_loss = 0
    total_examples = 0

    data_loader_gen = loader_generator(data, target_edge_types, batch_size, shuffle=True)

    for edge_type, loader in data_loader_gen:
        pbar = tqdm(loader, desc=f"Training {edge_type[1]}", leave=False)

        for batch in pbar:
            batch = batch.to(device)
            optimizer.zero_grad()

            edge_label_index = batch[edge_type].edge_label_index
            edge_label = batch[edge_type].edge_label

            with torch.amp.autocast('cuda'):
                # Forward (Encoder đã đóng băng sẽ không tính gradient)
                out = model(batch.x_dict, batch.edge_index_dict, edge_type, edge_label_index)
                loss = F.binary_cross_entropy_with_logits(out, edge_label)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item() * edge_label.size(0)
            total_examples += edge_label.size(0)

        del loader, pbar
        gc.collect()

    return total_loss / (total_examples + 1e-6)

@torch.no_grad()
def evaluate(model, data, device, target_edge_types, batch_size):
    model.eval()
    preds = []
    ground_truths = []

    data_loader_gen = loader_generator(data, target_edge_types, batch_size, shuffle=False)

    for edge_type, loader in data_loader_gen:
        pbar = tqdm(loader, desc=f"Validating {edge_type[1]}", leave=False)
        for batch in pbar:
            batch = batch.to(device)

            if not hasattr(batch[edge_type], 'edge_label_index') or batch[edge_type].edge_label_index.numel() == 0:
                continue

            edge_label_index = batch[edge_type].edge_label_index
            edge_label = batch[edge_type].edge_label

            with torch.amp.autocast('cuda'):
                out = model(batch.x_dict, batch.edge_index_dict, edge_type, edge_label_index)
                out = torch.sigmoid(out)

            preds.append(out.cpu().numpy())
            ground_truths.append(edge_label.cpu().numpy())

        del loader, pbar

    if len(preds) == 0: return 0.0
    final_labels = np.concatenate(ground_truths)
    final_preds = np.concatenate(preds)
    ap = 0
    f1 = 0
    try:
        ap = average_precision_score(final_labels, final_preds)
        f1 = f1_score(final_labels, (final_preds > 0.5).astype(int))
    except:
        pass
    auc = roc_auc_score(final_labels, final_preds )
    return auc, ap, f1

def update_training_history(file_path, new_data):
    history = []
    if os.path.exists(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                history = json.load(f)
                if not isinstance(history, list): history = [history]
        except: history = []

    if isinstance(new_data, list): history.extend(new_data)
    else: history.append(new_data)

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(history, f, indent=4, ensure_ascii=False)

# --- 4. MAIN EXECUTION: DECODER TUNING ---

def run_decoder_tuning():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🚀 RUNNING PHASE 3: RECOMMENDATION DECODER TUNING on {device}")
    MODEL_PATH = 'data_output/models/model_phase2_calibrated.pt'
    # 1. Load Data
    data = get_or_prepare_data()

    # 2. Split Data (Chỉ quan tâm Human-Hub)
    train_data, val_data, test_data, target_edges = prepare_decoder_splits(data)

    # 3. Cấu hình Phase 3
    config = {
        'hidden_dim': 256,
        'batch_size': 256, # Tăng Batch Size lên vì Encoder đã Freeze (ít tốn RAM hơn)
        'lr': 0.01,       # LR cao hơn để train MLP nhanh
        'epochs': 20,      # Train kỹ
        'dropout': 0.4
    }

    # 4. Khởi tạo Model
    print("Initializing Model...", end=" ")
    model = LinkPredictionModel(config['hidden_dim'], OUTPUT_DIM, train_data, config['dropout']).to(device)

    # --- QUAN TRỌNG: LOAD TRỌNG SỐ TỪ PHASE TRƯỚC ---
    print("\n📥 Loading Pre-trained Encoder weights...")
    try:
        # Load weights, bỏ qua strict để tránh lỗi nếu shape decoder khác nhau
        state_dict = torch.load(MODEL_PATH, map_location=device, weights_only=False)
        model.load_state_dict(state_dict, strict=False)
        print("✅ Loaded Pre-trained Weights successfully!")
    except Exception as e:
        print(f"⚠️ Warning: Could not load weights. Starting from scratch? Error: {e}")

    # --- QUAN TRỌNG: ĐÓNG BĂNG ENCODER ---
    print("🔒 FREEZING ENCODER (GNN)...")
    for param in model.gnn.parameters():
        param.requires_grad = False
    for param in model.encoder.parameters():
        param.requires_grad = False
    print("Encoder Frozen. Only Decoders will be updated.")
    params = []
    added_keys = set()
    for src, rel, dst in target_edges:
        key = f"__{rel}__"
        if key in model.decoders and key not in added_keys:
            params.extend(list(model.decoders[key].parameters()))
            added_keys.add(key)

    if not params:
        print("❌ Error: No decoder parameters found to train!")
        return

    optimizer = torch.optim.Adam(params, lr=config['lr'])
    scaler = torch.amp.GradScaler('cuda')
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-5)

    # 6. Training Loop
    history = {"epoch": [], "loss": [], "val_auc": []}
    best_val_auc = 0.0
    rec_model_path = str(MODEL_PATH).replace(".pt", "_rec_tuned.pth")
    rec_history_path = str(TRAINING_HISTORY_PATH).replace(".json", "_rec.json")

    print(f"\nStart Training Decoders for {config['epochs']} epochs...")

    for epoch in range(1, config['epochs'] + 1):
        # Train
        loss = train_epoch(model, train_data, optimizer, device, target_edges, scaler, config['batch_size'])

        # Validate
        val_auc, ap, f1 = evaluate(model, val_data, device, target_edges, config['batch_size'])

        scheduler.step()

        # Logging
        history["epoch"].append(epoch)
        history["loss"].append(loss)
        history["val_auc"].append(val_auc)
        try:
            history["ap"].append(ap)
            history["f1"].append(f1)
        except:
            pass
        print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Rec Val AUC: {val_auc:.4f} | LR: {optimizer.param_groups[0]['lr']:.5f}")

        # Save Best
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            repo = ModelRepository(rec_model_path)
            repo.save_model(model)

        # Clean VRAM
        torch.cuda.empty_cache()
        gc.collect()
        update_training_history(rec_history_path, history)

    # 7. Final Test
    print("\n🏁 Evaluating on Test Set...")
    if os.path.exists(rec_model_path):
        model.load_state_dict(torch.load(rec_model_path))
        test_auc, test_ap, test_f1 = evaluate(model, test_data, device, target_edges, config['batch_size'])
        history['test_auc'] = test_auc
        try:
            history['test_ap'] = test_ap
            history['test_f1'] = test_f1
        except:
            pass
        print(f"FINAL RECOMMENDATION TEST AUC: {test_auc:.4f}")

    update_training_history(rec_history_path, history)

import torch
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score

# Import lại các module cần thiết từ code cũ của bạn
# (Đảm bảo bạn đã import đủ các Class: LinkPredictionModel, GraphSAGE, InteractionMLP...)
from config.settings import MODEL_PATH, PYG_DATA_PATH, OUTPUT_DIM

def get_all_target_edges(data):
    """Copy lại hàm chọn cạnh bạn đã dùng ở Phase 3"""
    target_edges = []
    for edge_type in data.edge_types:
        src, rel, dst = edge_type
        # Lấy tất cả cạnh xuôi từ Human (như bạn đã cấu hình Phase 3)
        if src == 'human' and not rel.startswith('rev_'):
            target_edges.append(edge_type)

    rev_target_edges = []
    for (src, rel, dst) in target_edges:
        rev_rel = f"rev_{rel}"
        rev_target_edges.append((dst, rev_rel, src))
    return target_edges, rev_target_edges

def run_final_evaluation():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🔍 STARTING FINAL EVALUATION on {device}")

    # 1. LOAD DATA & SPLIT (Phải giống hệt lúc train Phase 3 để Test set đúng)
    print("1. Loading Data...")
    data = get_or_prepare_data()

    # Lấy danh sách cạnh đã train Phase 3
    target_edges, rev_target_edges = get_all_target_edges(data)

    print("2. Splitting Data (Re-creating Test Set)...")
    # Cấu hình Split giống hệt Phase 3
    transform = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.05,
        is_undirected=False,
        edge_types=target_edges,
        rev_edge_types=rev_target_edges,
        disjoint_train_ratio=0.2,
        add_negative_train_samples=False
    )
    # Chúng ta chỉ cần test_data
    _, _, test_data = transform(data)

    # 2. INIT MODEL
    print("3. Initializing Model Architecture...")
    # Lưu ý: hidden_dim và dropout phải khớp config lúc train
    model = LinkPredictionModel(hidden_channels=256, out_channels=OUTPUT_DIM, data=test_data, dropout=0.4).to(device)

    # 3. LOAD WEIGHTS PHASE 3
    # trỏ đúng vào file model phase 3 mà bạn đã lưu
    phase3_path = 'data_output/models/model_phase2_calibrated_rec_tuned.pth'
    # Hoặc nếu bạn lưu đè lên MODEL_PATH thì dùng MODEL_PATH

    print(f"4. Loading Weights from: {phase3_path}")
    try:
        model.load_state_dict(torch.load(phase3_path, map_location=device, weights_only= False))
        print("✅ Weights Loaded Successfully!")
    except Exception as e:
        print(f"❌ Error loading weights: {e}")
        return

    # 4. RUN DETAILED EVALUATION
    model.eval()
    print("\n" + "="*50)
    print(f"{'RELATIONSHIP':<30} | {'AUC':<8} | {'AP':<8}")
    print("="*50)

    all_preds = []
    all_labels = []

    # Duyệt qua từng loại cạnh để đánh giá riêng
    for et in target_edges:
        edge_name = et[1]

        # Kiểm tra xem có cạnh test không
        if (et not in test_data.edge_types) or \
           (not hasattr(test_data[et], 'edge_label_index')) or \
           (test_data[et].edge_label_index.numel() == 0):
            continue

        # Tạo Loader nhỏ cho Test
        lbl_index = test_data[et].edge_label_index
        lbl_ones = torch.ones(lbl_index.size(1), dtype=torch.float32)

        loader = LinkNeighborLoader(
            test_data,
            num_neighbors=[50, 30],
            edge_label_index=(et, lbl_index),
            edge_label=lbl_ones,
            neg_sampling_ratio=1.0, # Test 1 thật : 1 giả
            batch_size=128,         # Batch an toàn
            shuffle=False,
            num_workers=0
        )

        preds = []
        ground_truths = []

        for batch in tqdm(loader, desc=f"Testing {edge_name}", leave=False):
            batch = batch.to(device)
            with torch.no_grad(): # Tắt gradient cho nhẹ
                with torch.amp.autocast('cuda'):
                    out = model(batch.x_dict, batch.edge_index_dict, et, batch[et].edge_label_index)
                    scores = torch.sigmoid(out)

            preds.append(scores.cpu().numpy())
            ground_truths.append(batch[et].edge_label.cpu().numpy())

        if len(preds) > 0:
            edge_preds = np.concatenate(preds)
            edge_labels = np.concatenate(ground_truths)

            # Tính chỉ số
            auc = roc_auc_score(edge_labels, edge_preds)
            ap = average_precision_score(edge_labels, edge_preds)
            f1 = f1_score(edge_labels, (edge_labels > 0.5).astype(int))
            print(f"{edge_name:<30} | {auc:.4f}   | {ap:.4f} | {f1:.4f}")

            all_preds.append(edge_preds)
            all_labels.append(edge_labels)
        del loader
    # 5. TỔNG KẾT
    print("="*50)
    if len(all_preds) > 0:
        final_preds = np.concatenate(all_preds)
        final_labels = np.concatenate(all_labels)
        total_auc = roc_auc_score(final_labels, final_preds)
        print(f"🏆 OVERALL MODEL AUC: {total_auc:.4f}")
    print("="*50)

if __name__ == "__main__":
    run_final_evaluation()

🔍 STARTING FINAL EVALUATION on cuda
1. Loading Data...
REPO: Đang tải Processed Data...
2. Splitting Data (Re-creating Test Set)...
3. Initializing Model Architecture...
4. Loading Weights from: data_output/models/model_phase2_calibrated_rec_tuned.pth
✅ Weights Loaded Successfully!

RELATIONSHIP                   | AUC      | AP      


Testing acted_in:   0%|          | 0/721 [00:00<?, ?it/s]

acted_in                       | 0.7507   | 0.7977 | 1.0000


Testing active_in:   0%|          | 0/3 [00:00<?, ?it/s]

active_in                      | 0.7298   | 0.7866 | 1.0000


Testing active_in:   0%|          | 0/16 [00:00<?, ?it/s]

active_in                      | 0.7541   | 0.8155 | 1.0000


Testing adheres_to:   0%|          | 0/13 [00:00<?, ?it/s]

adheres_to                     | 0.7450   | 0.8061 | 1.0000


Testing advisor_of:   0%|          | 0/231 [00:00<?, ?it/s]

In [1]:
from config.settings import PYG_DATA_PATH
from infrastructure.repositories import PyGDataRepository
repo = PyGDataRepository(PYG_DATA_PATH)
data = repo.load_data()

REPO: Đang tải Processed Data...


In [7]:
spouse = data['human', 'spouse', 'human'].edge_index

In [21]:
src = spouse[0]
dst = spouse[1]

In [23]:
adj = zip(src,dst)

In [32]:
src_id = 1853535
dst_id = 278455
mask = (spouse[0] == dst_id) & (spouse[1] == src_id)

In [33]:
mask.any()

tensor(False)

In [ ]:
for et in data.edge_types:
    print(et, data[et].edge_index.size(1))

In [ ]:
import torch
from torch_geometric.utils import coalesce

def remove_duplicate_edges(data):
    """
    Loại bỏ các cạnh trùng lặp trong đồ thị (In-place modification).
    Hỗ trợ cả HeteroData và Data chuẩn.
    """
    print("--- BẮT ĐẦU XÓA CẠNH TRÙNG (Deduplication) ---")

    # 1. Xử lý cho HeteroData (Đồ thị dị thể)
    if hasattr(data, 'edge_types'):
        total_removed = 0
        for edge_type in data.edge_types:
            # Lấy edge_index cũ
            edge_index = data[edge_type].edge_index
            if edge_index.numel() == 0: continue

            old_count = edge_index.size(1)

            # Lấy số lượng node để tính toán sparse matrix (giúp coalesce nhanh hơn)
            # src_type, _, dst_type = edge_type
            # num_nodes = max(data[src_type].num_nodes, data[dst_type].num_nodes)
            # (Hoặc để None để nó tự tính max index, hơi chậm hơn xíu nhưng an toàn)

            # THẦN CHÚ COALESCE
            # args: edge_index, edge_attr (None nếu không có), sort_by_row
            new_edge_index, _ = coalesce(edge_index, None, sort_by_row=True)

            # Cập nhật lại vào data
            data[edge_type].edge_index = new_edge_index

            new_count = new_edge_index.size(1)
            diff = old_count - new_count

            if diff > 0:
                print(f"✅ {edge_type}: Đã xóa {diff} cạnh trùng ({old_count} -> {new_count})")
                total_removed += diff

        if total_removed == 0:
            print("-> Đồ thị sạch, không có cạnh trùng nào.")
        else:
            print(f"-> TỔNG CỘNG ĐÃ XÓA: {total_removed} cạnh.")

    # 2. Xử lý cho Data thường (Homogeneous)
    else:
        old_count = data.edge_index.size(1)
        data.edge_index, _ = coalesce(data.edge_index, None, sort_by_row=True)
        new_count = data.edge_index.size(1)
        print(f"-> Đã xóa {old_count - new_count} cạnh trùng.")

    return data

In [ ]:
data = remove_duplicate_edges(data)

In [ ]:
from torch_geometric.transforms import ToUndirected

data = ToUndirected(merge=False)(data)


In [ ]:
from torch_geometric.utils import sort_edge_index, coalesce
import torch
def process_human_edges(data):
    print("--- Processing Human Edges ---")

    # 1. Xóa employed_by (Như đã bàn)
    if ('human', 'employed_by', 'human') in data.edge_types:
        del data['human', 'employed_by', 'human']
        print("Dropped 'employed_by'")

    # 2. Gộp partner vào spouse
    if ('human', 'partner', 'human') in data.edge_types:
        # Lấy edges
        partner_index = data['human', 'partner', 'human'].edge_index
        spouse_index = data['human', 'spouse', 'human'].edge_index

        print(f"Merging: Spouse ({spouse_index.size(1)}) + Partner ({partner_index.size(1)})")

        # A. Nối lại (Concatenate)
        merged_index = torch.cat([spouse_index, partner_index], dim=1)

        # B. COALESCE (Thần chú quan trọng nhất)
        # Hàm này làm 2 việc:
        # 1. Sắp xếp lại (Sort) theo node nguồn.
        # 2. Gộp các cạnh trùng nhau (Remove Duplicates).
        # num_nodes là số lượng node Human (cần thiết để tính toán sparse)
        num_humans = data['human'].num_nodes
        merged_index, _ = coalesce(merged_index, None, num_nodes=num_humans, sort_by_row=True)

        # C. Cập nhật lại vào data
        data['human', 'spouse', 'human'].edge_index = merged_index

        # D. Xóa partner
        del data['human', 'partner', 'human']
        print(f"-> New Spouse size: {merged_index.size(1)}")

    # 3. (Optional) Sắp xếp lại tất cả các loại cạnh khác cho chắc ăn
    # Thao tác này rất nhanh và giúp Loader chạy mượt hơn
    if ('human', 'student_of', 'human') in data.edge_types and \
       ('human', 'advisor_of', 'human') in data.edge_types:

        # Lấy edges
        student_index = data['human', 'student_of', 'human'].edge_index
        advisor_index = data['human', 'advisor_of', 'human'].edge_index

        print(f"Merging: Advisor ({advisor_index.size(1)}) + Student ({student_index.size(1)})")

        # A. ĐẢO CHIỀU (FLIP) cạnh student_of
        # student_index đang là [Row=Student, Col=Teacher]
        # Ta lật lại thành [Row=Teacher, Col=Student] để khớp với logic advisor_of
        flipped_student_index = torch.stack([student_index[1], student_index[0]], dim=0)

        # B. Nối lại
        merged_index = torch.cat([advisor_index, flipped_student_index], dim=1)

        # C. COALESCE (Sắp xếp + Xóa trùng)
        num_humans = data['human'].num_nodes
        merged_index, _ = coalesce(merged_index, None, num_nodes=num_humans, sort_by_row=True)

        # D. Cập nhật và Xóa
        data['human', 'advisor_of', 'human'].edge_index = merged_index
        del data['human', 'student_of', 'human']

        print(f"-> New Advisor size: {merged_index.size(1)}")

    for et in data.edge_types:
        data[et].edge_index = sort_edge_index(data[et].edge_index)

    return data

In [ ]:
data = process_human_edges(data)

In [ ]:
for et in data.edge_types:
    if 'rev_' in et[1]:
        del data[et]

In [ ]:
from core.ai import GraphDataProcessor
import pandas as pd
nodes = pd.read_parquet('data_output/cleaned/nodes.parquet', engine='pyarrow')
edges = pd.read_parquet('data_output/cleaned/edges.parquet', engine='pyarrow')
processor = GraphDataProcessor()
processor.run(nodes, edges)

# Loading data

In [ ]:
#data = ToUndirected(merge=False)(data)

In [ ]:
#feature.save_data(data,mapping)

# Training